In [ ]:
from utils import list_images, simulate_cytoplasm
from tqdm import tqdm
from cellpose import models, core, io
import re
import numpy as np
from tifffile import imread, imwrite
from skimage.color import rgb2gray
from collections import defaultdict
from pathlib import Path
from skimage.measure import regionprops_table
import napari
import pandas as pd

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose-SAM and Spotiflow models
model = models.CellposeModel(gpu=True)

In [ ]:
data_folder = r"Z:\MariiaY\Evos Anna nanoparticles\Ex7_neg_LNPs_.2025-11-27-11-47-57"

In [ ]:
images = list_images(data_folder, format="tif")
images

In [ ]:
# === YOUR INPUTS ===
files = images

output_dir = Path("./processed_tiffs")
output_dir.mkdir(parents=True, exist_ok=True)


# Regex to extract well, frame, channel
pattern = re.compile(
    r'.*_(?P<well>[A-Z]\d{2})f(?P<frame>\d{2})d(?P<channel>\d)\.TIF$',
    re.IGNORECASE
)

groups = defaultdict(dict)
for f in files:
    m = pattern.match(f)
    if m:
        groups[(m.group('well'), m.group('frame'))][int(m.group('channel'))] = f


def grayscale_skimage(img):
    # Case 1: TIFF loads as (3, H, W)
    if img.ndim == 3 and img.shape[0] == 3 and img.shape[-1] != 3:
        img = np.moveaxis(img, 0, -1)  # → (H, W, 3)
    
    # Case 2: Already (H, W, 3) → fine
    if img.ndim == 3 and img.shape[-1] == 3:
        gray01 = rgb2gray(img)  # float64 in [0,1]
    
    else:
        raise ValueError(f"Unexpected image shape {img.shape}")

    # Restore original dtype
    if np.issubdtype(img.dtype, np.integer):
        maxval = np.iinfo(img.dtype).max
        gray = (gray01 * maxval).astype(img.dtype)
    else:
        gray = gray01.astype(img.dtype)

    return gray


# Build & save stacks
for (well, frame), channels in tqdm(groups.items()):
    expected = [1, 2, 4]
    if not all(c in channels for c in expected):
        print(f"Skipping incomplete {well}f{frame}")
        continue

    slices = []
    for c in expected:
        rgb = imread(channels[c], series=None)
        gray = grayscale_skimage(rgb)
        slices.append(gray)

    stack = np.stack(slices, axis=0)
    out_path = output_dir / f"{well}f{frame}.tif"
    imwrite(out_path, stack)

    print("Saved:", out_path)


In [ ]:
processed_img_path = "./processed_tiffs"
processed_images = list_images(processed_img_path, format="tif")

In [ ]:
img = imread(processed_images[14])

In [ ]:
field_of_view = Path(processed_images[14]).stem.split("f")[1]
well_id = Path(processed_images[14]).stem.split("f")[0]

# Create a dictionary containing all image descriptors
descriptor_dict = {"well_id": well_id, "FOV": field_of_view}

print(field_of_view, well_id)

In [ ]:
# Predict nuclei labels using CellposeSAM and simulate cytosol using pyclesperanto
nuclei_labels, flows, styles = model.eval(img[1], niter=1000) # need to check the arguments
cytoplasm_labels = simulate_cytoplasm(nuclei_labels, dilation_radius=3)

In [ ]:
# Visualize results in Napari
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img)
viewer.add_labels(nuclei_labels)
viewer.add_labels(cytoplasm_labels)

In [ ]:
props_list = []

# Extract nuclei avg_int
props = regionprops_table(label_image=nuclei_labels,
                        intensity_image=img[0],
                        properties=["label", "intensity_mean"])

# Convert to dataframe
props_df = pd.DataFrame(props)

# Rename intensity_mean column to indicate the specific image
props_df.rename(columns={"intensity_mean": f"nuclei_avg_int"}, inplace=True)

# Append each props_df to props_list
props_list.append(props_df)


# Extract cytoplasm avg_int
props = regionprops_table(label_image=cytoplasm_labels,
                        intensity_image=img[0],
                        properties=["label", "intensity_mean"])

# Convert to dataframe
props_df = pd.DataFrame(props)

# Rename intensity_mean column to indicate the specific image
props_df.rename(columns={"intensity_mean": f"cytoplasm_avg_int"}, inplace=True)

# Append each props_df to props_list
props_list.append(props_df)

In [ ]:
# Initialize the df with the first df in the list
props_df = props_list[0]
# Start looping from the second df in the list
for df in props_list[1:]:
    props_df = props_df.merge(df, on="label")

# Add each key-value pair from descriptor_dict to props_df at the specified position
insertion_position = 0
for key, value in descriptor_dict.items():
    props_df.insert(insertion_position, key, value)
    insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

props_df